In [1]:
import pickle
from collections import Counter

import numpy as np
import pandas as pd
from river import metrics, optim, reco

In [2]:
df = pd.read_pickle("../data/trainset.pkl")

In [3]:
trainset = df[["user_id", "anime_id", "rating"]].to_dict(orient="records")

In [4]:
del df

In [6]:
metric = metrics.MAE() + metrics.RMSE()

In [7]:
# Using tutorial parameters
# https://github.com/online-ml/river/blob/main/docs/examples/matrix-factorization-for-recommender-systems/part-1.ipynb
biased_mf_params = {
    "n_factors": 10,
    "bias_optimizer": optim.SGD(0.025),
    "latent_optimizer": optim.SGD(0.05),
    "weight_initializer": optim.initializers.Zeros(),
    "latent_initializer": optim.initializers.Normal(mu=0.0, sigma=0.1, seed=73),
    "l2_bias": 0.0,
    "l2_latent": 0.0,
}

model = reco.BiasedMF(**biased_mf_params)

In [8]:
for data in trainset:
    y_pred = model.predict_one(user=data["user_id"], item=data["anime_id"])
    metric = metric.update(data["rating"], y_pred)
    model = model.learn_one(
        user=data["user_id"], item=data["anime_id"], y=data["rating"]
    )

In [9]:
metric

MAE: 0.934269, RMSE: 1.235888

In [10]:
with open("../data/model.pkl", "wb") as f:
    pickle.dump(model, f)